In [36]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [3]:
conn.execute("DROP TABLE IF EXISTS especies_nativas_endemicas")
conn.execute("""
    CREATE TABLE especies_nativas_endemicas AS
    SELECT * FROM especies_nativas
    UNION ALL
    SELECT * FROM especies_endemicas
""")

In [4]:
conn.execute("DROP TABLE IF EXISTS gbif_nativas_endemicas")
conn.execute("""
    CREATE TABLE gbif_nativas_endemicas AS
    SELECT *
    FROM gbif
    WHERE species IN (
        SELECT DISTINCT(scientificName_y)
        FROM especies_nativas_endemicas
    )
""").df()

,Count
0,22378


In [5]:
conn.execute("""
    UPDATE gbif_nativas_endemicas
    SET namePublishedInYear = -1
    WHERE namePublishedInYear IS NULL
""").df()

,Count
0,22364


In [7]:
conn.execute("""ALTER TABLE gbif_nativas_endemicas ALTER COLUMN namePublishedInYear SET DATA TYPE INTEGER; """).df()

,Success


In [ ]:
sql="""
SELECT
    g.species,

    -- red list: pega apenas 1 valor (ou null)
    FIRST(e.redListCategory) AS redListCategory,

    CASE 
        WHEN MAX(g.year) < 1989 THEN 0
        ELSE 1
    END AS ultimo_ano_coleta,

    CASE                   
        WHEN MAX(g.namePublishedInYear) < 1989 THEN 0
        ELSE 1
    END AS ano_descricao,

    date_diff(
        'day',
        MIN(MAKE_DATE(
            CAST(COALESCE(g.year, 1) AS INTEGER),
            CAST(COALESCE(g.month, 1) AS INTEGER),
            CAST(COALESCE(g.day, 1) AS INTEGER)
        )),
        MAX(MAKE_DATE(
            CAST(COALESCE(g.year, 1) AS INTEGER),
            CAST(COALESCE(g.month, 1) AS INTEGER),
            CAST(COALESCE(g.day, 1) AS INTEGER)
        ))
    ) AS diferenca_data_coleta,

    CASE 
        WHEN COUNT(DISTINCT e.land_regions) > 1 THEN 0
        ELSE 1
    END AS endemica,

    COUNT(g.species) AS total_ocorrencias,

FROM gbif_nativas_endemicas g
LEFT JOIN especies_nativas_endemicas e
    ON e.scientificName_y = g.species

GROUP BY g.species;
"""
conn.execute(sql).df()

,species,redListCategory,ultimo_ano_coleta,ano_descricao,diferenca_data_coleta,endemica,total_ocorrencias
0,Piper nudilimbum,Least Concern,1,0,734841,0,588
1,Piper pilirameum,Least Concern,1,0,738252,1,152
2,Piper schiedeanum,Least Concern,1,0,728847,0,120
3,Piper insignilaminum,Vulnerable,1,0,7643,1,12
4,Piper skutchii,Endangered,0,0,8308,1,4
...,...,...,...,...,...,...,...
86,Piper crassinervium,Least Concern,1,0,739117,0,13392
87,Piper augustum,Least Concern,1,0,739214,0,11596
88,Piper phytolaccifolium,Least Concern,1,0,735932,0,3080
89,Piper reticulatum,Least Concern,1,0,739305,0,15318


In [ ]:
df.to_csv("clf.csv", sep=";", quoting=2, index=False, encoding="utf-8")

In [7]:
conn.close()